## Preface



We need some particular versions of the following modules;



In [ ]:
!pip install -r requirements.txt

Add a string for your username here:



In [ ]:
user = "ligon"

## USDA Food Central DataBase



The USDA maintains a database of nutritional information, where
  different kinds of food are identified by an FDC number.  They do
  not provide any data on prices.  

To look up nutritional information, use api provided by the USDA at
[https://fdc.nal.usda.gov/](https://fdc.nal.usda.gov/).   You&rsquo;ll need to sign up for a
free api key (see directions on page), then add that key here:



In [1]:
apikey = {'ligon':"inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN"}

## Data on Prices



Here&rsquo;s an effort to describe some different kinds of food, along with
data on food prices.  This is all just based on a trip I took to the
grocery store, except that I&rsquo;ve used the USDA database to look up FDC
numbers.  

Also note that some kinds of foods need to have unit weights (in
grams) supplied under &ldquo;Units&rdquo;; e.g., extra large eggs are taken to
each weigh 56g.  These conversions can also often be found on the USDA
FDC website.  Othertimes not&#x2014;I still need to weigh a crumpet.

Food is purchased in particular units (gallons, pounds, grams).  And
in some cases the natural units are things like donuts or eggs, in
which case we may need to define our  own units (see the example of
&ldquo;xl\_egg&rdquo; below).

| Food|Quantity|Units|Price|Date|Location|FDC|
|---|---|---|---|---|---|---|
| Milk, 2% fat|1|gallon|4.99|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|336075|
| Eggs, extra large|12|xl\_egg|3.59|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|747997|
| Crumpets|6|crumpet|3.19|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|547313|
| Bananas|1|pound|3.15|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|173944|
| Carrots, Organic|2|pound|2.29|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|170393|
| Cauliflower|2.51|pound|4.24|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|169986|
| Endive, Red|1.26|pound|6.27|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|168412|
| Figs, black mission|1|pound|4.98|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|438223|
| Leeks, Organic|1|pound|1.29|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|169246|
| Lettuce, Little Gem|1|pound|5.98|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|342618|
| Mushrooms, King Oyster|1|pound|12|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|342623|
| Onion, yellow|1|pound|0.39|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|170000|
| Orange juice|0.5|gallon|8.98|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|414575|
| Parsnip|1|pound|1.98|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|170417|
| Potato, marble mix|1|pound|2.59|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|170032|
| Rhubarb|1|pound|1.84|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|167758|
| Potato, russet|10|pound|2.98|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|170030|
| Squash, Zucchini|1|pound|1.49|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|169291|

And here&rsquo;s a little code to help look up FDC codes for foods of
different descriptions.



In [1]:
import fooddatacentral

fooddatacentral.fdc_search(apikey[user],"")

## Price information



The code below allows us to collect data on different kinds of food
with their prices from google spreadsheets.



### Using data from Google Sheets



Rather than creating & uploading `csv` files we can
also write code to find files over the internet.  Here we supply code
which allows one to input data on prices in a format similar to that
above (i.e., a spreadsheet with seven columns, labeled Food, Quantity,
Units, Price, Date, Location, and FDC).

Using google sheets in this way requires you establish some
credentials.   Follow Step 1 in [Turn on the API](https://developers.google.com/sheets/api/quickstart/python#step_1_turn_on_the_api_name).



In [1]:
# ID of sheet and name of worksheet for google sheets.   
# Easiest if these are public (or use service accounts)

SHEETs = [# Stigler's foods, modern prices
          ("https://docs.google.com/spreadsheet/ccc?key=1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A","Table 2"), 
         ]

### Compile data on food prices



In [1]:
import fooddatacentral
import pandas as pd
import warnings

DFs = []
#for csv in CSVs:  # Uncomment to use a list of csv files as inputs
#  DFs.append(pd.read_csv(csv,dtype=str))

try:
    if len(SHEETs):
        for ID, RANGE_NAME in SHEETs:
           try:
               if "docs.google.com" in ID:
                   sheet = "%s&output=csv" % ID
               else:
                   sheet = "https://docs.google.com/spreadsheet/ccc?key=%s&output=csv" % ID
               DFs.append(pd.read_csv(sheet))
           except ParserError:
               warnings.warn("Can't read sheet at https://docs.google.com/spreadsheets/d/%s.\nCheck Sharing settings, so that anyone with link can view?" % ID)
except NameError: # SHEETS not defined?
    pass

df = pd.concat(DFs,ignore_index=True,sort=False)
# Some columns which ought to be numeric are actually str; convert them
df['Price'] = df['Price'].astype(float)
df['Quantity'] = df['Quantity'].astype(float)
df['FDC'] = df['FDC'].astype(int)

df

### Look up nutritional information for foods



Now we have a list of foods with prices.  Do lookups on USDA database
to get nutritional information.



In [1]:
D = {}
count = 0
for food in  df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fooddatacentral.fdc_report(apikey[user],FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

D = pd.DataFrame(D,dtype=float)

D

## Units & Prices



Now, the prices we observe can be for lots of different quantities and
units.  The FDC database basically wants everything in either hundreds
of grams (hectograms) or hundreds of milliliters (deciliters).  

Sometimes this conversion is simple; if the price we observe is for
something that weighs two kilograms, that&rsquo;s just 20 hectograms.
Different systems of weights and volumes are also easy; a five pound
bag of flour is approximately 22.68 hectograms.  

Othertimes things are more complicated.  If you observe the price of a
dozen donuts, that needs to be converted to hectograms, for example.  

A function `fdc_units` in the [fdc](fooddatacentral.py) module accomplishes this conversion
for many different units, using the `python` [pint module](https://pint.readthedocs.io/en/latest/).  A file
[./Data/food\_units.txt](Data/food_units.txt) can be edited to deal with odd cases such as
donuts, using a format described in the `pint` [documentation](https://pint.readthedocs.io/en/latest/defining.html). 

Here&rsquo;s an example of the usage of `fooddatacentral.fdc_units`:



In [1]:
import fooddatacentral

# Try your own quantities and units.
# If units are missing try adding to ./Data/food_units.txt

print(fooddatacentral.fdc_units(5,'lbs'))
print(fooddatacentral.fdc_units(1,'gallon'))
print(fooddatacentral.fdc_units(2,'tea_bag'))
print(fooddatacentral.fdc_units(12,'donut'))

Now, use the `fdc_units` function to convert all foods to either
deciliters or hectograms, to match FDC database:



In [1]:
# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fooddatacentral.fdc_units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food')['FDC Price'].min()

Prices.head()

## Dietary Requirements



We&rsquo;ve figured out some foods we can buy, the nutritional content of
those foods, and  the price of the foods.  Now we need to say
something about nutritional requirements.   Our data for this is based
on  US government recommendations available at
[https://www.dietaryguidelines.gov/sites/default/files/2019-05/2015-2020_Dietary_Guidelines.pdf](https://www.dietaryguidelines.gov/sites/default/files/2019-05/2015-2020_Dietary_Guidelines.pdf).
Note that we&rsquo;ve tweaked the nutrient labels to match those in the FDC
data.

We&rsquo;ve broken down the requirements into three different tables.  The
first is *minimum* quantities that we need to  satisfy.  For example,
this table tells us that a 20 year-old female needs at least 46 grams
of protein per day.

| Nutrition|Source|C 1-3|F 4-8|M 4-8|F 9-13|M 9-13|F 14-18|M 14-18|F 19-30|M 19-30|F 31-50|M 31-50|F 51+|M 51+|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Energy|---|1000|1200|1400|1600|1800|1800|2200|2000|2400|1800|2200|1600|2000|
| Protein|RDA|13|19|19|34|34|46|52|46|56|46|56|46|56|
| Fiber, total dietary|---|14|16.8|19.6|22.4|25.2|25.2|30.8|28|33.6|25.2|30.8|22.4|28|
| Folate, DFE|RDA|150|200|200|300|300|400|400|400|400|400|400|400|400|
| Calcium, Ca|RDA|700|1000|1000|1300|1300|1300|1300|1000|1000|1000|1000|1200|1000|
| Carbohydrate, by difference|RDA|130|130|130|130|130|130|130|130|130|130|130|130|130|
| Iron, Fe|RDA|7|10|10|8|8|15|11|18|8|18|8|8|8|
| Magnesium, Mg|RDA|80|130|130|240|240|360|410|310|400|320|420|320|420|
| Niacin|RDA|6|8|8|12|12|14|16|14|16|14|16|14|16|
| Phosphorus, P|RDA|460|500|500|1250|1250|1250|1250|700|700|700|700|700|700|
| Potassium, K|AI|3000|3800|3800|4500|4500|4700|4700|4700|4700|4700|4700|4700|4700|
| Riboflavin|RDA|0.5|0.6|0.6|0.9|0.9|1|1.3|1.1|1.3|1.1|1.3|1.1|1.3|
| Thiamin|RDA|0.5|0.6|0.6|0.9|0.9|1|1.2|1.1|1.2|1.1|1.2|1.1|1.2|
| Vitamin A, RAE|RDA|300|400|400|600|600|700|900|700|900|700|900|700|900|
| Vitamin B-12|RDA|0.9|1.2|1.2|1.8|1.8|2.4|2.4|2.4|2.4|2.4|2.4|2.4|2.4|
| Vitamin B-6|RDA|0.5|0.6|0.6|1|1|1.2|1.3|1.3|1.3|1.3|1.3|1.5|1.7|
| Vitamin C, total ascorbic acid|RDA|15|25|25|45|45|65|75|75|90|75|90|75|90|
| Vitamin E (alpha-tocopherol)|RDA|6|7|7|11|11|15|15|15|15|15|15|15|15|
| Vitamin K (phylloquinone)|AI|30|55|55|60|60|75|75|90|120|90|120|90|120|
| Zinc, Zn|RDA|3|5|5|8|8|9|11|8|11|8|11|8|11|
| Vitamin D|RDA|600|600|600|600|600|600|600|600|600|600|600|600|600|

This next table specifies *maximum* quantities.  Our 20 year-old
female shouldn&rsquo;t have more than 2300 milligrams of sodium per day.
Note that we can also add constraints here on nutrients that also
appear above.  For example, here we&rsquo;ve added upper limits on Energy,
as we might do if we were trying to lose weight.

| Nutrition|Source|C 1-3|F 4-8|M 4-8|F 9-13|M 9-13|F 14-18|M 14-18|F 19-30|M 19-30|F 31-50|M 31-50|F 51+|M 51+|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Sodium, Na|UL|1500|1900|1900|2200|2200|2300|2300|2300|2300|2300|2300|2300|2300|
| Energy|---|1500|1600|1800|2000|2200|2200|2500|2400|2600|2200|2400|1800|2400|

Finally, we have some odd constraints given in this final table.
Mostly the items given don&rsquo;t correspond to items in the FDC data
(e.g., copper), but in some cases it may be possible to match things
up.  We can&rsquo;t use these without some additional work.

| Nutrition|Source|C 1-3|F 4-8|M 4-8|F 9-13|M 9-13|F 14-18|M 14-18|F 19-30|M 19-30|F 31-50|M 31-50|F 51+|M 51+|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Carbohydrate, % kcal|AMDR|45-65|45-65|45-65|45-65|45-65|45-65|45-65|45-65|45-65|45-65|45-65|45-65|45-65|
| Added sugars, % kcal|DGA|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|
| Total fat, % kcal|AMDR|30-40|25-35|25-35|25-35|25-35|25-35|25-35|20-35|20-35|20-35|20-35|20-35|20-35|
| Saturated fat, % kcal|DGA|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|<10%|
| Linoleic acid, g|AI|7|10|10|10|12|11|16|12|17|12|17|11|14|
| Linolenic acid, g|AI|0.7|0.9|0.9|1|1.2|1.1|1.6|1.1|1.6|1.1|1.6|1.1|1.6|
| Copper, mcg|RDA|340|440|440|700|700|890|890|900|900|900|900|900|900|
| Manganese, mg|AI|1.2|1.5|1.5|1.6|1.9|1.6|2.2|1.8|2.3|1.8|2.3|1.8|2.3|
| Selenium, mcg|RDA|20|30|30|40|40|55|55|55|55|55|55|55|55|
| Choline, mg|AI|200|250|250|375|375|400|550|425|550|425|550|425|550|

-   **Notes on Source:** In each of these tables, RDA = Recommended
    Dietary Allowance, AI = Adequate Intake, UL = Tolerable Upper
    Intake Level, AMDR = Acceptable Macronutrient Distribution
    Range, DGA = 2015-2020 Dietary Guidelines recommended limit; 14
    g fiber per 1,000 kcal = basis for AI for fiber.



In [1]:
# Choose sex/age group:
group = "F 19-30"

# Define *minimums*
bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition')[group]

# Define *maximums*
bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition')[group]

## Putting it together



Here we take the different pieces of the puzzle we&rsquo;ve developed and
put them together in the form of a linear program we can solve.



In [1]:
from  scipy.optimize import linprog as lp
import numpy as np

tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros

c = Prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(c.index.tolist()).intersection(D.columns.tolist()))
c = c[use]

# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[c.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([-Amin,Amax])

b = pd.concat([-bmin,bmax]) # Note sign change for min constraints

# Now solve problem!
result = lp(c, A, b, method='interior-point')

# Put back into nice series
diet = pd.Series(result.x,index=c.index)

print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))
print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.

tab = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())